In [12]:
import pandas as pd
from io import StringIO
from IPython.display import display

# 输入课程数据
data = """
课程号	课序号	课程名	开课系	可选容量	报名总人数	学位课报名人数	非学位课报名人数
82558001	210	文献检索与论文写作	图书馆	300	124	1,5,118	0,0,0
65990342	200	人工智能基础	国际研究生院	100	40	4,4,27	3,0,2
60420123	203	实验设计与数据处理	数学系	40	46	5,20,21	0,0,0
65990061	202	设计思维与实践	国际研究生院	30	34	1,4,29	0,0,0
65990082	200	交叉创新实践课——生命的数字逻辑	国际研究生院	20	34	0,20,13	1,0,0
60680021	205	自然辩证法概论	马克思主义学院	110	106	4,13,89	0,0,0
70240123	200	语音信号数字处理	计算机系	60	82	3,29,50	0,0,0
94200012	201	博士生英语	语言中心	36	22	10,4,8	0,0,0
60240103	200	大数据分析（B）	计算机系	100	201	133,16,47	2,2,1
"""

# 读取数据
df = pd.read_csv(StringIO(data), sep='\t')

# 提取各志愿人数
df[['第一志愿', '第二志愿', '第三志愿']] = df['学位课报名人数'].str.split(',', expand=True).astype(int)

# 计算录取概率
def calculate_probability(row):
    capacity = row['可选容量']
    first_choice = row['第一志愿']
    second_choice = row['第二志愿']
    third_choice = row['第三志愿']

    # 第一志愿的录取概率
    if first_choice <= capacity:
        first_choice_prob = 1.0
    else:
        first_choice_prob = capacity / first_choice

    # 第二志愿的录取概率
    remaining_capacity_after_first = max(0, capacity - first_choice)
    if second_choice <= remaining_capacity_after_first:
        second_choice_prob = 1.0
    else:
        if remaining_capacity_after_first > 0:
            second_choice_prob = remaining_capacity_after_first / second_choice
        else:
            second_choice_prob = 0

    # 第三志愿的录取概率
    remaining_capacity_after_second = max(0, remaining_capacity_after_first - second_choice)
    if third_choice <= remaining_capacity_after_second:
        third_choice_prob = 1.0
    else:
        if remaining_capacity_after_second > 0:
            third_choice_prob = remaining_capacity_after_second / third_choice
        else:
            third_choice_prob = 0

    return pd.Series([first_choice_prob, second_choice_prob, third_choice_prob], index=['第一志愿概率', '第二志愿概率', '第三志愿概率'])

# 应用计算概率函数
df[['第一志愿概率', '第二志愿概率', '第三志愿概率']] = df.apply(calculate_probability, axis=1)

# 分类课程
# 第一类：第一志愿人数超过可选容量
category1 = df[df['第一志愿'] > df['可选容量']]

# 第二类：第一志愿人数不足但第一二志愿人数之和超过可选容量
category2 = df[(df['第一志愿'] <= df['可选容量']) & ((df['第一志愿'] + df['第二志愿']) > df['可选容量'])]

# 第三类：第一二志愿人数之和不足但三志愿人数总和超过可选容量
category3 = df[(df['第一志愿'] + df['第二志愿'] <= df['可选容量']) & ((df['第一志愿'] + df['第二志愿'] + df['第三志愿']) > df['可选容量'])]

# 输出结果
print("第一类：第一志愿人数超过可选容量的课程：")
display(category1[['课程号', '课序号', '课程名', '可选容量', '报名总人数', '第一志愿', '第一志愿概率']].head())

print("\n第二类：第一志愿人数不足但第一二志愿人数之和超过可选容量的课程：")
display(category2[['课程号', '课序号', '课程名', '可选容量', '报名总人数', '第一志愿', '第二志愿', '第一志愿概率', '第二志愿概率']].head())

print("\n第三类：第一二志愿人数之和不足但三志愿人数总和超过可选容量的课程：")
display(category3[['课程号', '课序号', '课程名', '可选容量', '报名总人数', '第一志愿', '第二志愿', '第三志愿', '第一志愿概率', '第二志愿概率', '第三志愿概率']].head())

第一类：第一志愿人数超过可选容量的课程：


,课程号,课序号,课程名,可选容量,报名总人数,第一志愿,第一志愿概率
8,60240103,200,大数据分析（B）,100,201,133,0.75188



第二类：第一志愿人数不足但第一二志愿人数之和超过可选容量的课程：


,课程号,课序号,课程名,可选容量,报名总人数,第一志愿,第二志愿,第一志愿概率,第二志愿概率



第三类：第一二志愿人数之和不足但三志愿人数总和超过可选容量的课程：


,课程号,课序号,课程名,可选容量,报名总人数,第一志愿,第二志愿,第三志愿,第一志愿概率,第二志愿概率,第三志愿概率
2,60420123,203,实验设计与数据处理,40,46,5,20,21,1.0,1.0,0.714286
3,65990061,202,设计思维与实践,30,34,1,4,29,1.0,1.0,0.862069
4,65990082,200,交叉创新实践课——生命的数字逻辑,20,34,0,20,13,1.0,1.0,0.000000
6,70240123,200,语音信号数字处理,60,82,3,29,50,1.0,1.0,0.560000
